In [1]:
# импортим нужные библиотеки
import pandas as pd
import numpy as np
from pandas_datareader import data as pdr
import yfinance as yfin
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

<h4>1) Скачиваем данные об акциях, например Теслы, и смотрим на них

In [2]:
yfin.pdr_override()

data = pdr.get_data_yahoo(['TSLA'], start='2022-01-01', end='2023-01-01')['Adj Close'].reset_index()
data

[*********************100%%**********************]  1 of 1 completed


,Date,Adj Close
0,2022-01-03,399.926666
1,2022-01-04,383.196655
2,2022-01-05,362.706665
3,2022-01-06,354.899994
4,2022-01-07,342.320007
...,...,...
246,2022-12-23,123.150002
247,2022-12-27,109.099998
248,2022-12-28,112.709999
249,2022-12-29,121.820000


<h4>2) Посчитаем разницу между текущим днем и предыдущим за каждый день

In [3]:
return_df = pd.DataFrame()
return_df['Date'] = data['Date']
return_df['TSLA'] = data['Adj Close'].pct_change()
return_df

,Date,TSLA
0,2022-01-03,NaN
1,2022-01-04,-0.041833
2,2022-01-05,-0.053471
3,2022-01-06,-0.021523
4,2022-01-07,-0.035447
...,...,...
246,2022-12-23,-0.017551
247,2022-12-27,-0.114089
248,2022-12-28,0.033089
249,2022-12-29,0.080827


<h4>3) Посчитаем кумулятивный доход и стандартное отклонение

In [4]:
cumulative_return = (data['Adj Close'][len(data) - 1] / data['Adj Close'][0]) - 1
print(cumulative_return)

std = return_df['TSLA'].std() * np.sqrt(len(data))
print(std)

-0.6919935310711032
0.6520248269402337


<h4>4) Считаем коэффицент Шарпа (Sharpe ratio) по формуле:</h4>

$$
Sharpe = \frac{Expected Return - Risk Free Rate}{Standard Deviation}
$$
<br><b>Return</b> и <b>std</b> мы нашли, остался <b>risk-free rate</b>.<br><b>RFR</b> определяется, как правило, по доходности государственных облигаций или по проценту на вклад в банке за исследуемый период. <br><br>
Возьмем RFR США на начало 2022 года = 0.8% (доходность облиг. на начало 2022)<br><br>
Ссылка на данные о доходности годовых облигаций США: https://ru.investing.com/rates-bonds/u.s.-1-year-bond-yield


In [5]:
rfr = 0.8 / 100
sharpe = (cumulative_return - rfr) / std
sharpe

-1.0735688307391191

Как видим, вложиться в Теслу в 2022 году было бы плохим решением, так как коэффицент Шарпа меньше нуля.

<h4>5) Теперь напишем функцию в общем виде для нескольких акций, любых дат и rfr:

In [6]:
def sharpe_ratio(stocks : list, start_date : str, end_date : str, rfr : float):
    data = pd.DataFrame()
    return_df = pd.DataFrame()
    cumulative_return = dict()
    std = dict()
    sharpe_ratio = pd.DataFrame()
    for name in stocks:
        data[name] = pdr.get_data_yahoo([name], start=start_date, end=end_date)['Adj Close']
        return_df[name] = data[name].pct_change()
        cumulative_return[name] = (data[name][len(data) - 1] / data[name][0]) - 1
        std[name] = return_df[name].std() * np.sqrt(len(data))
        sharpe_ratio[name] = [(cumulative_return[name] - rfr) / std[name]]
    data['Overall'] = data.apply(lambda x: sum(x[name] for name in stocks), axis=1)
    return_df['Overall'] = data['Overall'].pct_change()
    cumulative_return['Overall'] = (data['Overall'][len(data) - 1] / data['Overall'][0]) - 1
    std['Overall'] = return_df['Overall'].std() * np.sqrt(len(data))
    sharpe_ratio['Overall'] = [(cumulative_return['Overall'] - rfr) / std['Overall']]
    return sharpe_ratio

<h5>Посмотрим пример на нескольких акциях:

In [7]:
names = ['TSLA', 'AAPL', 'GOOG', 'META', 'AMZN', 'TSM', 'FORD', 'MCD']

Sharpe_ratio за 2023 год

In [8]:
sharpe_2023 = sharpe_ratio(names, start_date='2023-01-01', end_date='2023-11-01', rfr=0.047)
sharpe_2023

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


,TSLA,AAPL,GOOG,META,AMZN,TSM,FORD,MCD,Overall
0,1.627732,1.712057,1.197875,3.537344,1.577339,0.469932,-1.034741,-0.300676,2.086651


За 2022 год

In [9]:
sharpe_2022 = sharpe_ratio(names, start_date='2022-01-01', end_date='2022-12-31', rfr=0.008)
sharpe_2022

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


,TSLA,AAPL,GOOG,META,AMZN,TSM,FORD,MCD,Overall
0,-1.073569,-0.814564,-1.022443,-1.0156,-1.029736,-1.056037,-0.64551,-0.024393,-1.320826


Объединим датасеты по годам

In [10]:
total_sharpe = pd.concat([sharpe_2022, sharpe_2023])
total_sharpe['year'] = ['2022', '2023']
total_sharpe = total_sharpe.set_index('year')
total_sharpe

,TSLA,AAPL,GOOG,META,AMZN,TSM,FORD,MCD,Overall
year,,,,,,,,,
2022,-1.073569,-0.814564,-1.022443,-1.015600,-1.029736,-1.056037,-0.645510,-0.024393,-1.320826
2023,1.627732,1.712057,1.197875,3.537344,1.577339,0.469932,-1.034741,-0.300676,2.086651


Построим график

In [17]:
fig = px.bar(total_sharpe, barmode='group', title='Sharpe ratio comparison')
fig.update_layout(yaxis_title='Sharpe ratio')
fig.show()

На графике видно, что в 2023 году все представленные акции, за исключением FORD, показали себя лучше, чем в 2022.